In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests

import json
from pprint import pprint

from config import nyt_api_key

import pandas as pd

from sqlalchemy import create_engine
import pymysql

In [ ]:
movies = ["War", "Dog", "Money", "Games", "Love", "Car", "Future", "Girls", "Tomorrow", "Nine"]
url = 'http://www.omdbapi.com/?apikey=f6f49e7d&s='


movie_title = [];
year = [];
imdbID = [];


for movie in movies:
    for i in range(1,11):
        
        movie_data = requests.get(url + movie + "&page=" + str(i)).json()

        for x in range(10):

            movie_title.append(movie_data['Search'][x]['Title'])
            year.append(movie_data['Search'][x]['Year'])
            imdbID.append(movie_data['Search'][x]['imdbID'])

In [ ]:
movies_df = pd.DataFrame({
    "title_omdb": movie_title,
    "omdb_year": year,
    "omdb_id": imdbID
})

movies_df = movies_df.reset_index().rename(columns={'index':'id'})

movies_df.head()

In [ ]:
url = "https://api.nytimes.com/svc/movies/v2/reviews/search.json?"

In [ ]:
encoded_omdb_movies = []

for i in range(0, len(movies_df['title_omdb'])):
    encoded_omdb_movies.append('%27' + movies_df['title_omdb'][i].replace(' ','%20') + '%27')

In [ ]:
title_nyt = []
byline_nyt = []
open_date_nyt = []

for encode in encoded_omdb_movies:
    try:
        query_url = url + "api-key=" + nyt_api_key + "&query=" + encode
        review = requests.get(query_url).json()
        title_nyt.append(review['results'][0]['display_title'])
        byline_nyt.append(review['results'][0]['byline'])
        open_date_nyt.append(review['results'][0]['opening_date'])
    
    except (IndexError, KeyError) as e:
        title_nyt.append('Not found')
        byline_nyt.append('Not found')
        open_date_nyt.append('Not found')

In [ ]:
nyt_dict = {
    'title_nyt': title_nyt,
    'byline_nyt': byline_nyt,
    'open_date_nyt': open_date_nyt
}

In [ ]:
nyt_df = pd.DataFrame(nyt_dict)
nyt_df = nyt_df.reset_index().rename(columns={'index':'id'})

In [ ]:
nyt_df.head()

In [ ]:
pymysql.install_as_MySQLdb()

In [ ]:
rds_connection_string = "root:Hoosier36#^@127.0.0.1/movie_data_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
nyt_df.to_sql(name='nyt', con=engine, if_exists='append', index=False)

In [ ]:
movies_df.to_sql(name='omdb', con=engine, if_exists='append', index=False)